#NB Classifier

In [1]:
import pandas as pd
import csv

In [29]:
df = pd.read_csv("/content/drive/MyDrive/Truth_Seeker/cleaned_master_data.csv")
df

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,title,lemmatized_text,label,subject,text_length,title_length
0,0,0,0,\r\r\r\r\r\r\r\nDonald Trump Gets His A Handed...,yearold girl smacked donald trump amazingon sa...,0,General News,2566,79
1,1,1,1,\r\r\r\r\r\r\r\nExGOP Congressman Shreds Fello...,point every republican refuse spine stand dona...,0,General News,1925,104
2,2,2,2,\r\r\r\r\r\r\r\nTrump Gets STOMPED For Whining...,donald trump attacking president obama campaig...,0,General News,3389,106
3,3,3,3,\r\r\r\r\r\r\r\nWATCH Democratic Rep Delivers ...,massachusetts rep jim mcgovern brought house s...,0,General News,3358,117
4,4,4,4,AfterTrumpImplodes Hashtag Hilariously Imagin...,world like postdonald trump new hashtag aftert...,0,General News,3092,86
...,...,...,...,...,...,...,...,...,...
76472,76472,76472,76540,Es hat sich wie ein Virus verbreitet Wie Poliz...,den teilweise erschtternden berichten geben di...,1,Unknown,6171,79
76473,76473,76473,76541,Reichsbürger schlagen zu Sind Beamte für Reic...,gleich mehrere polizisten bayern und sachsenan...,1,Unknown,5998,71
76474,76474,76474,76542,The Duelist Martins Wuttkes neuer Tatort,film deutschland und russland der neue film de...,1,Unknown,6916,40
76475,76475,76475,76543,俄罗斯总统观注,franais english espaol,1,Unknown,2150,7


In [23]:
df.shape

(76477, 7)

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

In [31]:
df_filtered = df[['lemmatized_text', 'label']]

In [107]:
X= df_filtered['lemmatized_text']
y = df_filtered['label']

In [108]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [109]:
vectorizer = CountVectorizer(stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [110]:
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_vec, y_train)

MultinomialNB()

In [111]:
y_pred = nb_classifier.predict(X_test_vec)

In [112]:
nb = accuracy_score(y_test, y_pred)*100

In [113]:
print("Accuracy:", nb)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 80.70737447698745

Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.80      0.78      6439
           1       0.85      0.81      0.83      8857

    accuracy                           0.81     15296
   macro avg       0.80      0.81      0.80     15296
weighted avg       0.81      0.81      0.81     15296



In [49]:
doc = 'In a groundbreaking discovery, a group of scientists has reportedly found a cure for cancer that could save millions of lives. However, sources within the research community claim that the cure is being suppressed by powerful pharmaceutical companies who fear the loss of profits from cancer treatments. Despite growing support for the findings, the medical industry and government officials are said to be working together to keep the discovery under wraps.There\'s a cure for cancer, and it\’s been available for years, said an anonymous scientist who claims to have worked on the treatment. But the big drug companies don’t want the world to know about it because they\’d lose billions of dollars in revenue from chemotherapy drugs and radiation treatments.Medical experts around the world have cast doubt on these claims, calling it a conspiracy theory with no scientific backing. Yet, the story continues to circulate in alternative media outlets and among conspiracy theorists. Critics argue that the narrative is a part of a larger agenda to control the health sector and profit from people\'s suffering.'

In [50]:
doc = vectorizer.transform([doc])

In [51]:
nb_classifier.predict(doc)

array([1])

In [114]:
from sklearn.model_selection import GridSearchCV

param_grid = {'alpha': [0.0001, 0.5, 1, 2, 5, 10]}

grid_search = GridSearchCV(estimator=MultinomialNB(),param_grid=param_grid,scoring='accuracy',cv=5,verbose=1)

grid_search.fit(X_train_vec, y_train)

print("Best Cross-Validation Accuracy:", grid_search.best_score_*100)

best_model = grid_search.best_estimator_
y_pred_tuned = best_model.predict(X_test_vec)

tuned = accuracy_score(y_test, y_pred_tuned)*100
print("\nTuned Accuracy:", tuned)
print("\nClassification Report(Tuned):\n", classification_report(y_test, y_pred_tuned))

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Best Cross-Validation Accuracy: 82.52562466101084

Tuned Accuracy: 82.55753138075313

Classification Report(Tuned):
               precision    recall  f1-score   support

           0       0.78      0.82      0.80      6439
           1       0.86      0.83      0.85      8857

    accuracy                           0.83     15296
   macro avg       0.82      0.82      0.82     15296
weighted avg       0.83      0.83      0.83     15296



#SVM

In [52]:
from sklearn.svm import SVC

In [115]:
svm_classifier = SVC(class_weight='balanced')

In [116]:
df_sample = df_filtered.sample(n=1000, random_state=42)

In [117]:
X = df_sample['lemmatized_text']
y = df_sample['label']

In [118]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [119]:
vectorizer = CountVectorizer(stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [120]:
svm_classifier.fit(X_train_vec, y_train)

SVC(class_weight='balanced')

In [121]:
y_pred = svm_classifier.predict(X_test_vec)

In [122]:
svm = accuracy_score(y_test, y_pred)*100

In [123]:
print("Accuracy: ", svm)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy:  73.0

Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.68      0.67        81
           1       0.78      0.76      0.77       119

    accuracy                           0.73       200
   macro avg       0.72      0.72      0.72       200
weighted avg       0.73      0.73      0.73       200



In [124]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 1.5, 0.5],'kernel': ['linear', 'rbf'],'gamma': ['scale', 'auto']}

grid_search = GridSearchCV(svm_classifier, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train_vec, y_train)

print("Best parameters found: ", grid_search.best_params_)

best_svm_classifier = grid_search.best_estimator_

Best parameters found:  {'C': 1.5, 'gamma': 'scale', 'kernel': 'rbf'}


In [125]:
svm_tuned = accuracy_score(best_svm_classifier.predict(X_test_vec), y_test)*100

In [126]:
print("Accuracy:",svm_tuned)

Accuracy: 76.0


In [128]:
print("NB Accuracy:",tuned_nb)
print("SVM Accuracy:",svm_tuned)

NB Accuracy: 82.55753138075313
SVM Accuracy: 76.0
